<a href="https://colab.research.google.com/github/ErtKid/Machine_Learning_H3/blob/main/TP_ML_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Étape 1: Nettoyage des données**

**1.1 Imputer les valeurs manquantes**

Pour la colonne "Age", nous allons remplacer les valeurs manquantes par la médiane. Pour "Embarked", nous utiliserons le port le plus fréquent, qui est "S". Nous allons ignorer la colonne "Cabin" car elle a trop de valeurs manquantes.

In [ ]:
# Importer la bibliothèque pandas
import pandas as pd

# Charger les données
titanic_df = pd.read_csv('Dataset_Titanic/train.csv')

# Imputer les valeurs manquantes pour "Age" avec la médiane
titanic_df['Age'].fillna(titanic_df['Age'].median(), inplace=True)

# Imputer les valeurs manquantes pour "Embarked" avec 'S'
titanic_df['Embarked'].fillna('S', inplace=True)

# Supprimer la colonne "Cabin"
titanic_df.drop('Cabin', axis=1, inplace=True)


**1.2 Ingénierie des fonctionnalités**

Nous allons créer une nouvelle variable "TravelAlone" pour indiquer si le passager voyage seul ou avec sa famille. Ensuite, nous allons encoder les variables catégorielles.


In [ ]:
# Créer la variable "TravelAlone"
titanic_df['TravelAlone'] = (titanic_df['SibSp'] + titanic_df['Parch'] == 0).astype(int)

# Encoder les variables catégorielles
titanic_encoded = pd.get_dummies(titanic_df, columns=['Pclass', 'Embarked', 'Sex'], drop_first=True)


# **Étape 2: Sélection des caractéristiques**

Nous allons utiliser RFE (Recursive Feature Elimination) pour sélectionner les caractéristiques les plus importantes.

In [ ]:
# Importer les bibliothèques nécessaires
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Définir les variables indépendantes et la variable cible
X = titanic_encoded.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
y = titanic_encoded['Survived']

# Initialiser le modèle de régression logistique
model = LogisticRegression(max_iter=1000)


# Appliquer RFE
rfe = RFE(estimator=model, n_features_to_select=8)
fit = rfe.fit(X, y)

# Afficher les caractéristiques sélectionnées
print("Caractéristiques sélectionnées:", X.columns[fit.support_])


Caractéristiques sélectionnées: Index(['Age', 'SibSp', 'Parch', 'TravelAlone', 'Pclass_2', 'Pclass_3',
       'Embarked_S', 'Sex_male'],
      dtype='object')


# **Étape 3: Préparation et évaluation du modèle**

Nous allons maintenant préparer notre modèle en utilisant la régression logistique et évaluer ses performances.

In [ ]:
# Importer les bibliothèques nécessaires
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle
model.fit(X_train, y_train)

# Faire des prédictions
y_pred = model.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred)

print("Précision du modèle:", accuracy)
print("Log Loss:", logloss)


Précision du modèle: 0.7932960893854749
Log Loss: 7.450364108364997
